<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Set-up-environment-and-required-packages" data-toc-modified-id="Set-up-environment-and-required-packages-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Set up environment and required packages</a></span></li><li><span><a href="#Load-the-data" data-toc-modified-id="Load-the-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load the data</a></span></li><li><span><a href="#Partition-the-data-in-training-and-test-set" data-toc-modified-id="Partition-the-data-in-training-and-test-set-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Partition the data in training and test set</a></span></li><li><span><a href="#Build-the-Model" data-toc-modified-id="Build-the-Model-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Build the Model</a></span></li><li><span><a href="#Evaluate-on-the-test-set" data-toc-modified-id="Evaluate-on-the-test-set-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Evaluate on the test set</a></span><ul class="toc-item"><li><span><a href="#Confusion-Matrix" data-toc-modified-id="Confusion-Matrix-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Confusion Matrix</a></span></li><li><span><a href="#Classification-Accuracy" data-toc-modified-id="Classification-Accuracy-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Classification Accuracy</a></span></li><li><span><a href="#ROC-Curve" data-toc-modified-id="ROC-Curve-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>ROC Curve</a></span></li><li><span><a href="#Area-under-Curve-(AUC)" data-toc-modified-id="Area-under-Curve-(AUC)-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>Area under Curve (AUC)</a></span></li></ul></li><li><span><a href="#Controlling-the-complexity" data-toc-modified-id="Controlling-the-complexity-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Controlling the complexity</a></span></li></ul></div>

# Decision Trees in R

We're using the [rpart](https://www.rdocumentation.org/packages/rpart) package from R. 
There are alternative in implementations in R, for instance the [tree](https://www.rdocumentation.org/packages/tree) package.

The dataset is available from https://archive.ics.uci.edu/ml/datasets/Bank+Marketing.

## Set up environment and required packages

The package *tidyverse* includes *dplyr, tidyr, readr, ggplot2*

In [ ]:
options(warn=-1)

suppressPackageStartupMessages(library(tidyverse))
suppressPackageStartupMessages(library(caret))
suppressPackageStartupMessages(library(ROCR))
suppressPackageStartupMessages(library(rpart))
suppressPackageStartupMessages(library(rpart.plot))

## Load the data
We have 3 data sets
1. The full bank data set with more than 41.000 entries, quite unbalanced
2. A smaller subset - still unbalanced
3. A balanced sample of the full data set with ~ 9200 entries

In [ ]:
data_dir_default = "../data/"
data_sets = c("bank-full", "bank-10percent", "bank-balanced")

A little helper function for loading the different data sets

In [ ]:
read_data <- function(data_set, data_dir = data_dir_default) {
  data_set <- paste(data_dir, data_set, ".csv", sep='')
  read.csv(data_set)
}

Load the data

In [ ]:
bank_data <- read_data(data_sets[1])
cat("# data rows: ", nrow(bank_data), "- # features: ", ncol(bank_data), "\n")

## Partition the data in training and test set

Another helper

In [ ]:
partition_data <- function(data, prop = 0.8) {

  set.seed(4711)
  n <- nrow(data)
  n_train <- round(0.8 * n) 
  partition <- sample(1:n, n_train)
  
  first <-  data[partition,]
  second  <-  data[-partition,]
  
  list(first, second)
}

We'll use the standard 80/20 split

In [ ]:
partitions <- partition_data(bank_data)
train.df <- partitions[[1]]
test.df  <- partitions[[2]]

cat("Number of training samples :", nrow(train.df), "\n")
cat("Number of test samples     :", nrow(test.df), "\n")

## Build the Model

We're using the *rpart* routine with its default settings. *rpart* implements the CART algorithm with tree pruning.

In [ ]:
bank_model1 <- rpart(formula = y ~ ., 
                     data = train.df, 
                     method = "class")

rpart.plot(bank_model1)

Each node in the tree describes a splitting criterion. Also each node shows
 * the predicted class
 * the predicted probability of having value 'yes'
 * the percentage of observations in the node

## Evaluate on the test set
To get the predicted classes we need to call *predict.rpart* with *type="class"*, for a probability matrix with *type="prob"*

In [ ]:
predicted <- function(model, data) {
    predicted_class = predict(object = model,  
                                newdata = data,
                                type = "class")  

    predicted_probs = predict(object = model,  
                                newdata = data,   
                                type = "prob")
    predicted_probs_yes <- predicted_probs[,"yes"]
    
    return (list(predicted_class, predicted_probs_yes))
}

predicted_class_probs <- predicted(bank_model1, test.df)
predict.class <- predicted_class_probs[[1]]
predict.probs.yes <- predicted_class_probs[[2]]

### Confusion Matrix

In [ ]:
evaluation <- confusionMatrix(data = predict.class,       
                              reference = test.df$y)
print(evaluation)

### Classification Accuracy

In [ ]:
accuracy <- evaluation$overall["Accuracy"]
cat("Classification Accuracy : ", format(100*accuracy,digits = 4), "%\n")

### ROC Curve

In [ ]:
pred <- prediction(predict.probs.yes, test.df$y)
roc_perf <- performance(pred,"tpr","fpr")
plot(roc_perf, colorize=TRUE)

### Area under Curve (AUC)

In [ ]:
auc_perf <- performance(pred,"auc")
auc <- auc_perf@y.values[[1]]
cat("AUC :", auc)

## Controlling the complexity
*rpart* has a so-called *complexity parameter* that basically controls the tree pruning
(see http://cran.r-project.org/web/packages/rpart/vignettes/longintro.pdf, Section 4).
The default value for cp is 0.01. Let's build a more complex tree.

In [ ]:
bank_model2 <- rpart(formula = y ~ ., 
                     data = train.df, 
                     method = "class",
                     cp = 0.005)

rpart.plot(bank_model2)

Note that we have a deeper tree now. Contrary to the previous one this tree considers the *month* feature.

Let's do a few quantitive comparisons for different values of *cp* and different data sets.

In [ ]:
build_and_evaluate_model <- function(data_set, cp) {
    data <- read_data(data_set)
    partitions <- partition_data(data)
    train.df <- partitions[[1]]
    test.df  <- partitions[[2]]
    model <- rpart(formula = y ~ ., 
                   data = train.df, 
                   method = "class",
                   cp = cp)

    predicted_class_probs <- predicted(model, test.df)
    predict.class <- predicted_class_probs[[1]]
    predict.probs.yes <- predicted_class_probs[[2]]
    
    evaluation <- confusionMatrix(data = predict.class,       
                                  reference = test.df$y)
    
    accuracy <- evaluation$overall["Accuracy"]
    
    pred <- prediction(predict.probs.yes, test.df$y)
    auc_perf <- performance(pred,"auc")
    auc <- auc_perf@y.values[[1]]
    
    return(list(accuracy, auc))
}

In [ ]:
for (data_set in data_sets) {
  cat("\n=====", data_set, "=====\n\n")
  flush.console()
  for ( cp in c(0.01, 0.005, 0.001, 0.0005)) {
      measures <- build_and_evaluate_model(data_set, cp)
      cat("cp : ", cp, 
          "\taccuracy : ", format(100*measures[[1]],digits = 4), 
          "\tAUC : ", measures[[2]], 
          "\n")
      flush.console()
  }
}